In [3]:
import os
import openai
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents.models import Vector  

from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType

# Replace these with your own values, either in environment variables or directly here
AZURE_STORAGE_ACCOUNT = os.environ.get("AZURE_STORAGE_ACCOUNT") or "sttyf3apvjit5gg"
AZURE_STORAGE_CONTAINER = os.environ.get("AZURE_STORAGE_CONTAINER") or "content"
AZURE_SEARCH_SERVICE = os.environ.get("AZURE_SEARCH_SERVICE") or "cogsearch001"
AZURE_SEARCH_INDEX = os.environ.get("AZURE_SEARCH_INDEX") or "gptkb-tyf3apvjit5gg"
AZURE_OPENAI_SERVICE = os.environ.get("AZURE_OPENAI_SERVICE") or "openai002"
AZURE_OPENAI_GPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_GPT_DEPLOYMENT") or "davinci"
AZURE_OPENAI_CHATGPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_CHATGPT_DEPLOYMENT") or "gpt-35-turbo"
AZURE_OPENAI_CHATGPT_MODEL = os.environ.get("AZURE_OPENAI_CHATGPT_MODEL") or "gpt-35-turbo"
AZURE_OPENAI_EMB_DEPLOYMENT = os.environ.get("AZURE_OPENAI_EMB_DEPLOYMENT") or "text-embedding-ada-002"
AZURE_SEARCH_ADMIN_KEY="2HwNw2JAmUjp8lYFfiiIMeIFTPye8rHHq9ZYu3hc9YAzSeA9Vl5L"
KB_FIELDS_CONTENT = os.environ.get("KB_FIELDS_CONTENT") or "content"
KB_FIELDS_CATEGORY = os.environ.get("KB_FIELDS_CATEGORY") or "category"
KB_FIELDS_SOURCEPAGE = os.environ.get("KB_FIELDS_SOURCEPAGE") or "sourcepage"

# Use the current user identity to authenticate with Azure OpenAI, Cognitive Search and Blob Storage (no secrets needed, 
# just use 'az login' locally, and managed identity when deployed on Azure). If you need to use keys, use separate AzureKeyCredential instances with the 
# keys for each service
azure_credential = DefaultAzureCredential(exclude_shared_token_cache_credential = True)
search_credential = AzureKeyCredential(AZURE_SEARCH_ADMIN_KEY)
# Used by the OpenAI SDK
openai.api_base = f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com"
openai.api_version = "2023-05-15"

# Comment these two lines out if using keys, set your API key in the OPENAI_API_KEY environment variable and set openai.api_type = "azure" instead
openai.api_type = "azure_ad"
openai.api_key = azure_credential.get_token("https://cognitiveservices.azure.com/.default").token

# Set up clients for Cognitive Search and Storage
search_client = SearchClient(
    endpoint=f"https://{AZURE_SEARCH_SERVICE}.search.windows.net",
    index_name=AZURE_SEARCH_INDEX,
    credential=search_credential)


In [13]:
# Execute this cell multiple times updating user_input to accumulate chat history
user_input = "Does it cover dental?"

# Exclude category, to simulate scenarios where there's a set of docs you can't see
exclude_category = None


# Use Azure OpenAI to compute an embedding for the query
query_vector = openai.Embedding.create(engine=AZURE_OPENAI_EMB_DEPLOYMENT, input=search)["data"][0]["embedding"]

# Alternatively simply use the following if not using reranking with semantic search:
# search_client.search(search, 
#                      top=3, 
#                      vector=query_vector if query_vector else None, 
#                      top_k=50 if query_vector else None, 
#                      vector_fields="embedding" if query_vector else None)

filter = "category ne '{}'".format(exclude_category.replace("'", "''")) if exclude_category else None

vector = Vector(value=query_vector, k=3, fields="embedding")

r = search_client.search(None, 
                         filter=filter,
                         top=3,
                         vectors=[vector], 
)
results = [doc[KB_FIELDS_SOURCEPAGE] + ": " + doc[KB_FIELDS_CONTENT].replace("\n", "").replace("\r", "") for doc in r]
content = "\n".join(results)
print(content)
user_content = user_input + " \nSOURCES:\n" + content

chat_conversations.append({"role": USER, "content": user_content })

chat_completion = openai.ChatCompletion.create(
    deployment_id=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
    model=AZURE_OPENAI_CHATGPT_MODEL,
    messages=chat_conversations, 
    temperature=0.7, 
    max_tokens=1024, 
    n=1)
chat_content = chat_completion.choices[0].message.content
'''
reset user content to avoid sources in conversation history
add source as a single shot in query conversation
'''
chat_conversations[-1]["content"] = user_input
chat_conversations.append({"role":ASSISTANT, "content": chat_content})

print("\n-------------------\n")
for conversation in chat_conversations:
    print(conversation)

Northwind_Health_Plus_Benefits_Details-23.pdf:  This coverage includes medically necessary services for the relief of pain resulting from dental injury, as well as services for the administration of anesthesia in a facility. This coverage is subject to any limitations, copayments, and/or deductibles that are set forth in the plan.In order for services to be eligible for coverage, they must be performed by a dental professional who is licensed to practice dentistry in the state in which the services are provided. All services must be for the relief of pain resulting from dental injury, or for the administration of anesthesia in a facility.Dental InjuryNorthwind Health Plus covers medically necessary services for the relief of pain resulting from dental injury. This includes services such as extractions, fillings, root canals, and other services that are necessary to relieve pain caused by dental injury.Facility Anesthesia Northwind Health Plus covers services related to the administrati

In [35]:
search_client = SearchClient(
    endpoint=f"https://{AZURE_SEARCH_SERVICE}.search.windows.net",
    index_name="payroll_hr",
    credential=search_credential)


In [ ]:
r = search_client.search(search, 
                         filter=filter,
                         query_type=QueryType.SEMANTIC, 
                         query_language="en-us", 
                         query_speller="lexicon", 
                         semantic_configuration_name="default", 
                         top=3,
                         vectors=[query_vector] if query_vector else None, 
                         top_k=50 if query_vector else None, 
                         vector_fields="embedding" if query_vector else None)


In [ ]:
results = [doc[KB_FIELDS_SOURCEPAGE] + ": " + doc[KB_FIELDS_CONTENT].replace("\n", "").replace("\r", "") for doc in r]


In [28]:

vector = Vector(value=query_vector, k=3, fields="contentVector")
  


In [36]:
results = search_client.search(  
    search_text=None,  
    vectors= [vector],
    select=["id", "content"],
)  
text_content =""
for result in results:  
    text_content += f"{result['id']}\n{result['content']}\n"
text_content

"article_15\n16. ADP HR: Understanding Your Health Insurance Options \n\nTitle: ADP HR: Understanding Your Health Insurance Options\n\nAs an employee with payroll managed by ADP, you have access to a range of health insurance options that are designed to suit your needs and budget. If you’re new to the world of health insurance or just want to refresh your knowledge, this guide will help you understand your options and make an informed decision about your health care coverage.\n\n1. Know Your Options\n\nThe first step in understanding your health insurance options is to know what’s available. ADP offers several health insurance plans, ranging from basic coverage to comprehensive plans with additional benefits. These plans include PPOs, HMOs, HDHPs, and more. Each of these types of plans has different features, costs, and restrictions, so it’s important to understand your options before making a decision.\n\n2. Consider Your Needs\n\nWhen selecting a health insurance plan, consider your

In [ ]:
results = [doc[KB_FIELDS_SOURCEPAGE] + ": " + doc[KB_FIELDS_CONTENT].replace("\n", "").replace("\r", "") for doc in r]
content = "\n".join(results)
